In [460]:
import networkx as nx
import numpy as np
import random

def generate_strongly_connected_tap_graph(n_nodes=20, min_cap=100, max_cap=500, min_t0=1, max_t0=10, demand_pairs=80):
    # Шаг 1: Создаём минимальное остовное дерево (MST) в виде неориентированного графа
    T = nx.generators.trees.random_tree(n_nodes)
    
    # Шаг 2: Превращаем в сильно связный направленный граф
    G = nx.DiGraph()
    for u, v in T.edges():
        G.add_edge(u, v)
        G.add_edge(v, u)  # Делаем двустороннее ребро для сохранения связности

    # Шаг 3: Добавляем случайное количество дополнительных рёбер
    min_edges = G.number_of_edges()  # Минимальное число рёбер (остовное дерево)
    max_edges = min(n_nodes * (n_nodes - 1), n_nodes * (n_nodes - 1))  # Ограничение сверху
    target_edges = random.randint(min_edges, max_edges)  # Случайное число рёбер

    while G.number_of_edges() < target_edges:
        u, v = np.random.choice(n_nodes, size=2, replace=False)
        if not G.has_edge(u, v):
            G.add_edge(u, v)

    # Шаг 4: Назначаем capacity и t0
    for u, v in G.edges():
        G[u][v]['capacity'] = random.randint(min_cap, max_cap)
        G[u][v]['t0'] = random.uniform(min_t0, max_t0)

    # Шаг 5: Генерация demand matrix (случайный спрос между парами узлов)
    demand_matrix = np.zeros((n_nodes, n_nodes))
    nodes = list(G.nodes())
    for _ in range(random.randint((n_nodes ** 2) // 2, n_nodes ** 2)):
        src, dst = np.random.choice(nodes, size=2, replace=False)
        demand_matrix[(src, dst)] = random.randint(10, 500)  # случайный спрос

    # Проверяем, что граф действительно сильно связный
    assert nx.is_strongly_connected(G), "Ошибка: граф не сильно связный!"

    return G, demand_matrix

# Пример использования
G, demand_matrix = generate_strongly_connected_tap_graph()

# Вывод данных


In [461]:
G.adjacency

<bound method Graph.adjacency of <networkx.classes.digraph.DiGraph object at 0x0000021A798FA8D0>>

In [462]:
import numpy as np

def get_matrices_from_graph(G):
    n = len(G.nodes)
    capacity_matrix = np.zeros((n, n))
    t0_matrix = np.zeros((n, n))

    for u, v, data in G.edges(data=True):
        capacity_matrix[u, v] = data['capacity']
        t0_matrix[u, v] = data['t0']

    return capacity_matrix, t0_matrix

# Получаем матрицы
capacity_matrix, t0_matrix = get_matrices_from_graph(G)

# Вывод примера
print("Capacity matrix:\n", capacity_matrix)
print("t0 matrix:\n", t0_matrix)

Capacity matrix:
 [[  0.   0.   0.   0. 353.   0. 138.   0. 468.   0.   0. 117.   0.   0.
  434. 411.   0.   0. 493. 280.]
 [  0.   0.   0. 168. 347. 102. 437. 459. 158. 153. 389.   0.   0.   0.
  332.   0. 226.   0. 424.   0.]
 [177.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 497. 396.
  445. 498.   0. 141. 148. 234.]
 [  0.   0.   0.   0. 266. 412. 359. 407.   0. 333.   0. 314.   0.   0.
    0.   0. 406.   0. 410. 350.]
 [415.   0.   0.   0.   0.   0.   0. 461.   0.   0.   0.   0. 169.   0.
    0.   0.   0. 119. 359.   0.]
 [399. 374. 156.   0. 159.   0.   0. 238.   0.   0. 144. 265.   0. 398.
    0. 160.   0.   0. 388. 469.]
 [185.   0. 101. 487.   0. 371.   0.   0. 428.   0. 155. 266. 472.   0.
    0. 180. 351. 465. 112.   0.]
 [309.   0.   0.   0. 352. 194.   0.   0.   0.   0. 191.   0.   0.   0.
  403. 350.   0.   0. 180. 182.]
 [  0.   0.   0.   0.   0.   0. 105. 112.   0. 207. 487.   0.   0.   0.
    0.   0.   0.   0.   0. 211.]
 [  0.   0.   0. 195.   0. 306. 279. 

In [463]:
demand_matrix

array([[  0., 117., 385., 284.,   0., 369., 119., 470., 131., 106.,   0.,
        480.,   0.,  23., 278.,   0., 221.,   0., 463.,   0.],
       [  0.,   0., 120., 336., 118.,   0.,   0.,   0.,   0.,   0., 416.,
          0.,   0., 486.,   0.,   0., 446.,  69.,   0., 495.],
       [245.,  87.,   0., 428.,  66., 313.,   0., 354.,   0.,  87.,   0.,
        209.,  90.,   0.,  48.,   0., 480., 360., 300.,   0.],
       [411.,   0., 221.,   0.,   0., 375.,   0., 125., 455.,   0.,   0.,
          0., 100.,  92.,   0., 298.,   0., 403., 133.,   0.],
       [  0., 418., 269., 313.,   0., 424.,   0.,   0.,  94., 458.,   0.,
          0.,  93.,   0.,  85., 408., 456., 315.,   0.,   0.],
       [175., 353., 301.,  24., 356.,   0.,   0.,   0., 499.,   0.,   0.,
          0., 158.,   0., 253., 160., 216.,   0., 213.,   0.],
       [335.,  81.,   0.,   0., 196., 129.,   0., 401.,   0., 266.,   0.,
        277.,   0.,   0.,   0., 453., 224., 433., 332.,   0.],
       [  0., 198.,   0.,  68., 447.,   0

In [1]:
from utility import generate_alike_data_C


generate_alike_data_C('siouxfalls/net.tntp', 'siouxfalls/trips.tntp', 10000, 1, 0.5, 'data/data_sioux_c.csv')

c:\py\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/10000 [00:00<?, ?it/s]d:\projects\src\leblanc.py:13: RuntimeWarning: invalid value encountered in divide
  T_1 = T_0 * (1 + 0.15 * (X_1 / C)**4)
d:\projects\src\leblanc.py:18: RuntimeWarning: divide by zero encountered in divide
  C_calc = 1 / (C**4)
d:\projects\src\leblanc.py:36: RuntimeWarning: divide by zero encountered in divide
  dif_val = (X_2 - X_1) / X_1
d:\projects\src\leblanc.py:36: RuntimeWarning: invalid value encountered in divide
  dif_val = (X_2 - X_1) / X_1
100%|██████████| 10000/10000 [1:26:53<00:00,  1.92it/s]


In [1]:
from utility import generate_alike_data_num_od_pairs as gen

gen('siouxfalls/net.tntp', 'siouxfalls/trips.tntp', 10, 1.5, 0.5, 1, 0.5, 'data_sioux_5pairs_test.csv', 5)

c:\py\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/10 [00:00<?, ?it/s]d:\projects\src\leblanc.py:13: RuntimeWarning: invalid value encountered in divide
  T_1 = T_0 * (1 + 0.15 * (X_1 / C)**4)
d:\projects\src\leblanc.py:18: RuntimeWarning: divide by zero encountered in divide
  C_calc = 1 / (C**4)
d:\projects\src\leblanc.py:36: RuntimeWarning: divide by zero encountered in divide
  dif_val = (X_2 - X_1) / X_1
d:\projects\src\leblanc.py:36: RuntimeWarning: invalid value encountered in divide
  dif_val = (X_2 - X_1) / X_1
100%|██████████| 10/10 [00:00<00:00, 12.24it/s]


In [4]:
import pandas as pd

df = pd.read_csv('data_sioux_5pairs_test.csv')

# Загружаем данные (замени на свой путь или способ получения DataFrame)

# Выбираем столбцы d_1...d_576
d_columns = [f"D_{i}" for i in range(0, 576)]

# Считаем количество ненулевых значений в каждой строке
df["nonzero_count"] = (df[d_columns] != 0).sum(axis=1)

# Выводим первые строки с результатом
print(df[["nonzero_count"]].head())
 

   nonzero_count
0              5
1              5
2              5
3              5
4              5


In [5]:
from utility import generate_alike_data as gen

gen('siouxfalls/net.tntp', 'siouxfalls/trips.tntp', 10000, 1.5, 0.5, 1, 1, 'data/data_sioux_d.csv')

  0%|          | 0/10000 [00:00<?, ?it/s]d:\projects\src\leblanc.py:13: RuntimeWarning: invalid value encountered in divide
  T_1 = T_0 * (1 + 0.15 * (X_1 / C)**4)
d:\projects\src\leblanc.py:18: RuntimeWarning: divide by zero encountered in divide
  C_calc = 1 / (C**4)
d:\projects\src\leblanc.py:36: RuntimeWarning: divide by zero encountered in divide
  dif_val = (X_2 - X_1) / X_1
d:\projects\src\leblanc.py:36: RuntimeWarning: invalid value encountered in divide
  dif_val = (X_2 - X_1) / X_1
100%|██████████| 10000/10000 [1:20:52<00:00,  2.06it/s]


In [1]:
from utility import generate_alike_data_num_od_pairs as gen

gen('siouxfalls/net.tntp', 'siouxfalls/trips.tntp', 10000, 1.5, 0.5, 1, 0.5, 'data_sioux_5pairs_test.csv', 5)

c:\py\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/10000 [00:00<?, ?it/s]d:\projects\src\leblanc.py:13: RuntimeWarning: invalid value encountered in divide
  T_1 = T_0 * (1 + 0.15 * (X_1 / C)**4)
d:\projects\src\leblanc.py:18: RuntimeWarning: divide by zero encountered in divide
  C_calc = 1 / (C**4)
d:\projects\src\leblanc.py:36: RuntimeWarning: invalid value encountered in divide
  dif_val = (X_2 - X_1) / X_1
100%|██████████| 10000/10000 [13:42<00:00, 12.16it/s]


In [3]:
from utility import generate_alike_data_num_od_pairs as gen

gen('siouxfalls/net.tntp', 'siouxfalls/trips.tntp', 10000, 1.5, 0.5, 1, 0.5, 'data/data_sioux_100pairs.csv', 100)

  0%|          | 0/10000 [00:00<?, ?it/s]d:\projects\src\leblanc.py:13: RuntimeWarning: invalid value encountered in divide
  T_1 = T_0 * (1 + 0.15 * (X_1 / C)**4)
d:\projects\src\leblanc.py:18: RuntimeWarning: divide by zero encountered in divide
  C_calc = 1 / (C**4)
d:\projects\src\leblanc.py:36: RuntimeWarning: divide by zero encountered in divide
  dif_val = (X_2 - X_1) / X_1
d:\projects\src\leblanc.py:36: RuntimeWarning: invalid value encountered in divide
  dif_val = (X_2 - X_1) / X_1
100%|██████████| 10000/10000 [1:12:36<00:00,  2.30it/s]


In [3]:
from utility import generate_alike_data_num_od_pairs as gen
    
gen('siouxfalls/net.tntp', 'siouxfalls/trips.tntp', 10000, 1.5, 0.5, 1, 0.5, 'data/data_sioux_576pairs.csv', 576-24, fixed=True)

  0%|          | 0/10000 [00:00<?, ?it/s]d:\projects\src\leblanc.py:13: RuntimeWarning: invalid value encountered in divide
  T_1 = T_0 * (1 + 0.15 * (X_1 / C)**4)
d:\projects\src\leblanc.py:18: RuntimeWarning: divide by zero encountered in divide
  C_calc = 1 / (C**4)
d:\projects\src\leblanc.py:36: RuntimeWarning: divide by zero encountered in divide
  dif_val = (X_2 - X_1) / X_1
d:\projects\src\leblanc.py:36: RuntimeWarning: invalid value encountered in divide
  dif_val = (X_2 - X_1) / X_1
  0%|          | 11/10000 [00:05<1:24:02,  1.98it/s]


KeyboardInterrupt: 

In [3]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
from utility import generate_n_pairs
generate_n_pairs('data/data_sioux.csv', 80, 'data/data_sioux_80_2.csv')

10000it [1:12:31,  2.30it/s]


In [5]:
import torch

if torch.cuda.is_available():
    print("Доступен CUDA GPU")
    print(f"Количество GPU: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"[{i}] {torch.cuda.get_device_name(i)}")
        print(f"  Выделено памяти: {torch.cuda.memory_allocated(i) / 1024 ** 2:.2f} MB")
        print(f"  Зарезервировано памяти: {torch.cuda.memory_reserved(i) / 1024 ** 2:.2f} MB")
else:
    print("GPU не найден, используется CPU")

Доступен CUDA GPU
Количество GPU: 1
[0] NVIDIA GeForce RTX 3050 Laptop GPU
  Выделено памяти: 0.00 MB
  Зарезервировано памяти: 0.00 MB
